In [11]:
# Imports
from neo4j import GraphDatabase
import time

In [10]:
# Here we create the connection class to enable us to connect to our Neo4j database.
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()


In [5]:
# Constructing our Neo4j connection string.
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="12345678")

In [16]:
# Here we can send queries to our Neo4j database: Or create a new DB if none exists
# First we make sure that there are no leftover data in our DB.
conn.query("MATCH ()-[r]->() DELETE r")
conn.query("MATCH (n) DETACH DELETE n")
print('Neo4j database was succesfully purged of data')

[<Record n=<Node element_id='4:2413d9a0-ce39-460b-9601-c09d38e3cabb:0' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='4:2413d9a0-ce39-460b-9601-c09d38e3cabb:1' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='4:2413d9a0-ce39-460b-9601-c09d38e3cabb:2' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='4:2413d9a0-ce39-460b-9601-c09d38e3cabb:3' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='4:2413d9a0-ce39-460b-9601-c09d38e3cabb:4' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='4:2413d9a0-ce39-460b-9601-c09d38e3cabb:5' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='4:2413d9a0-ce39-460b-9601-c09d38e3cabb:6' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='4:2413d9a0-ce39-460b-9601-c09d38e3cabb:7' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='4:2413d9a0-ce39-460b-9601-c09d38e3cabb:8' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='4:2413d9

In [19]:
# Inorder to make the next step work we need to move a copy of the csv file named "games_stripped.csv" into the import folder
# of the Neo4j project.

# Vi kunne også bare ændre vores query så den tager den raw fil fra github nu, måske smartere og mindre opsætning for
# dem der skal review på peergrade.

In [17]:
# Then we populate the database with data using the following command.
conn.query("LOAD CSV WITH HEADERS FROM 'file:///games_stripped.csv' as row MERGE (g: Game{title: row.Title, rating: row.Rating, publishers: split(row.Team, ','), genres: split(row.Genres, ','), reviews: split(row.Reviews, ',')}) WITH g MATCH (g) UNWIND g.publishers as pub MERGE (p: Publisher{name: pub}) MERGE (g)-[r1: RELEASED_BY]-> (p) WITH g MATCH (g) UNWIND g.genres as gen MERGE (ge: Genre{name: gen}) MERGE (g)-[r2: HAS_A]-> (ge) WITH g MATCH (g) UNWIND g.reviews as rev MERGE (re: Review{name: rev}) MERGE (g)-[r3: HAS_A]-> (re)")

[]

In [12]:
# Here we will demostrate the K-Nearest Neighbors-(KKN) similarity algorithm.


